# Gold Silver Scraping

## Import Libraries

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

C:\Users\mazen\anaconda3\envs\Safty\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Make The Request From Yahoo Finance and Get The Table HTML

In [2]:
url = "https://finance.yahoo.com/quote/SI%3DF/history/?period1=967608000&period2=1745260511"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
r = requests.get(url, headers=headers)

c = r.text
soup=BeautifulSoup(c, "lxml")

table = soup.find("table" , {"class":"table yf-1jecxey noDl hideOnPrint"})

## Get The Headers of The Table

In [3]:
headers = table.find_all("th" , {"class":"yf-1jecxey"})

titles = []

for i in headers:
    titles.append(i.text.split("  ")[0])
    
df = pd.DataFrame(columns = titles)
df

,Date,Open,High,Low,Close,Adj Close,Volume


## Get The Content of The Table

In [4]:
rows_con = table.find_all("tr" , {"class":"yf-1jecxey"})
for i in rows_con[1:]:
    data = i.find_all("td" , {"class":"yf-1jecxey"})
    row = [tr.text for tr in data]
    new_row_df = pd.DataFrame([row], columns=df.columns)  # Ensure columns match
    df = pd.concat([df, new_row_df], ignore_index=True)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,"Apr 21, 2025",32.440,33.100,32.385,32.555,32.555,"46,173"
1,"Apr 17, 2025",32.335,32.580,32.090,32.420,32.420,97
2,"Apr 16, 2025",33.025,33.025,32.755,32.926,32.926,97
3,"Apr 15, 2025",32.227,32.227,32.227,32.227,32.227,45
4,"Apr 14, 2025",31.800,32.092,31.800,32.092,32.092,17


## Data Preprocessing

In [5]:
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,"Apr 21, 2025",32.440,33.100,32.385,32.555,32.555,"46,173"
1,"Apr 17, 2025",32.335,32.580,32.090,32.420,32.420,97
2,"Apr 16, 2025",33.025,33.025,32.755,32.926,32.926,97
3,"Apr 15, 2025",32.227,32.227,32.227,32.227,32.227,45
4,"Apr 14, 2025",31.800,32.092,31.800,32.092,32.092,17
5,"Apr 11, 2025",31.115,31.824,31.115,31.824,31.824,24
6,"Apr 10, 2025",31.030,31.050,30.671,30.671,30.671,4
7,"Apr 9, 2025",29.350,30.323,29.255,30.323,30.323,137
8,"Apr 8, 2025",29.765,30.320,29.592,29.592,29.592,64
9,"Apr 7, 2025",28.450,30.460,28.310,29.510,29.510,310


In [6]:
df.shape

(6183, 7)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6183 entries, 0 to 6182
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       6183 non-null   object
 1   Open       6183 non-null   object
 2   High       6183 non-null   object
 3   Low        6183 non-null   object
 4   Close      6183 non-null   object
 5   Adj Close  6183 non-null   object
 6   Volume     6183 non-null   object
dtypes: object(7)
memory usage: 338.3+ KB


## Date Format

In [8]:
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6183 entries, 0 to 6182
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       6183 non-null   datetime64[ns]
 1   Open       6183 non-null   object        
 2   High       6183 non-null   object        
 3   Low        6183 non-null   object        
 4   Close      6183 non-null   object        
 5   Adj Close  6183 non-null   object        
 6   Volume     6183 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 338.3+ KB


## Numeric Data Format

In [9]:
numbers = df.select_dtypes(include = ["object"]).columns
numbers

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [10]:
for col in numbers:
    df[col] = df[col].str.replace("," , "")
    df[col] = pd.to_numeric(df[col], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6183 entries, 0 to 6182
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       6183 non-null   datetime64[ns]
 1   Open       6183 non-null   float64       
 2   High       6183 non-null   float64       
 3   Low        6183 non-null   float64       
 4   Close      6183 non-null   float64       
 5   Adj Close  6183 non-null   float64       
 6   Volume     5449 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 338.3 KB


In [11]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2025-04-21,32.440,33.100,32.385,32.555,32.555,46173.0
1,2025-04-17,32.335,32.580,32.090,32.420,32.420,97.0
2,2025-04-16,33.025,33.025,32.755,32.926,32.926,97.0
3,2025-04-15,32.227,32.227,32.227,32.227,32.227,45.0
4,2025-04-14,31.800,32.092,31.800,32.092,32.092,17.0


In [12]:
df = df.drop(columns = ['Open', 'High', 'Low', 'Close', 'Volume'])

In [13]:
df.head()

,Date,Adj Close
0,2025-04-21,32.555
1,2025-04-17,32.420
2,2025-04-16,32.926
3,2025-04-15,32.227
4,2025-04-14,32.092


In [14]:
df = df.set_index('Date')

In [15]:
df = df.asfreq('b', method = 'ffill')

In [16]:
df.head()

,Adj Close
Date,
2000-08-30,4.930
2000-08-31,5.003
2000-09-01,5.004
2000-09-04,4.998
2000-09-05,4.998


In [17]:
df.tail()

,Adj Close
Date,
2025-04-15,32.227
2025-04-16,32.926
2025-04-17,32.420
2025-04-18,32.555
2025-04-21,32.555


# Save The Data In CSV File

In [19]:
df.to_csv("silver_close.csv")